## Import Library

In [3]:
from requests import get
from requests.compat import urljoin
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm
import pandas as pd
import re
import time


## 전처리 함수 정의하기

In [4]:
def preprocessing(d):
    d = d.lower()
    d = re.sub(r'[a-z0-9\-_.]{3,}@[a-z0-9\-_.]{3,}(?:[.]?[a-z]{2})+', ' ', d)
    d = re.sub(r'‘’ⓒ\'\"“”…=□*◆:/_]', ' ', d)
    d = re.sub(r'\s+', ' ', d)
    d = re.sub(r'^\s|\s$', '', d)
    d = re.sub(r'[\<br\>]','',d)
    d = re.sub(r'[<*>_="/]', '', d)
    return d

## Base URL 설정

### 페이지수를 볼 수 있는 네이버 뉴스 주소 (일반 뉴스는 .../main/main.naver?... 주소로 이루어짐)
'https://news.naver.com/main/list.naver?

mode=LSD&

mid=shm&

sid1=100&   -- 카테고리

date={date}& -- 상세카테고리

page={page}' -- 페이지


sid1 (카테고리) : 100(정치), 101(경제), 102(사회), 103(생활문화), 104(세계), 105(IT과학), 106(연예), 107(스포츠), 110(오피니언)

sid2 (상세카테고리) : 


In [5]:
base_url = 'https://news.naver.com/main/list.naver?mode=LSD&mid=shm&sid1=100&date={date}&page={page}'

In [6]:
# 예외 단어 처리 - 재사용하기 위해 컴파일로 미리 저장.
exclude_keywords = re.compile(r'(속보|포토)')

## 메타데이터 수집용 함수 정의

In [7]:
# 기사 콘텐츠 (본문) 수집기 함수 정의
# 작동 안 됨 -> 다음 단계에서 본문 전문 수집. 
# def fetch_article_content(article_url):
#     resp = get(article_url)
#     article_dom = BeautifulSoup(resp.text, 'html.parser')
#     content_tag = article_dom.select_one('div#articleBodyContents')
#     if content_tag:
#         return content_tag.get_text(strip=True)
#     return ''

# 기사 가져오는 함수
def fetch_articles(url):
    resp = get(url)
    dom = BeautifulSoup(resp.text, 'html.parser')
    articles = dom.select('.content ul.type06_headline li')
    article_data_list = []

    # DOM select로 얻어낸 html 반복문 실행하여 데이터 추출
    for article in articles:
        title_tag = article.find('dt', class_=None)
        title_text = title_tag.get_text(strip=True)
       
        # 제목내 예외단어 발견시 추출 건너뛰기 (넘어가기)
        if exclude_keywords.search(title_text):
            continue
    
        # 기사 링크 태그, 주소, 요약 태그, 기자 태그
        link_tag = title_tag.find('a')
        article_url = link_tag['href']
        summary_tag = article.find('span', class_='lede')
        source_tag = article.find('span', class_='writing')
        
        # 기사 콘텐츠 (본문) 수집기
        # content = fetch_article_content(article_url)

        # 기사별 메타데이터 딕셔너리 생성
        article_dict = {
            'title': title_text,
            'link': article_url,
            'summary': summary_tag.get_text(strip=True) if summary_tag else '',
            'source': source_tag.get_text(strip=True) if source_tag else '',
            # 'content': content
        }
        
        # 여러 기사에 대해 메타데이터 작성
        article_data_list.append(article_dict)

    return article_data_list, dom

# 다음 페이지 넘어가기 : 최대 10페이지 전까지 정의
def get_next_page(dom, current_page):
    if current_page < 10:
        next_page_tag = dom.select_one(f'.paging a[href*="page={current_page + 1}"]')
        if next_page_tag:
            return urljoin(base_url, next_page_tag['href'])
    return None

# 일자별 크롤링 함수
def generate_dates(start_date, end_date):
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += timedelta(days=1)

## 기사 메타데이터 (제목, 링크, 기자, 요약 등) 수집하기

In [8]:
# 기사 수집 시작 / 종료 일자 (ex 2022-05-10: 2022년 5월 10일자)
start_date = datetime.strptime('2024-04-23', '%Y-%m-%d')
end_date = datetime.strptime('2024-04-24', '%Y-%m-%d')

all_articles = []

for single_date in tqdm(generate_dates(start_date, end_date)):
    date_str = single_date.strftime('%Y%m%d')
    page = 1
    url = base_url.format(date=date_str, page=page)
    
    while url and page <= 10:
        article_data_list, dom = fetch_articles(url)
        all_articles.extend(article_data_list)
        url = get_next_page(dom, page)
        page += 1
        time.sleep(2)  # 서버에 부담을 주지 않도록 잠시 대기


2it [00:49, 24.82s/it]


## 기사 본문 전문 수집하기

In [9]:
# 메타데이터 DataFrame 생성
df1 = pd.DataFrame(all_articles)

In [ ]:
# 메타데이터 중복처리 & 백업
df1.drop_duplicates(ignore_index=True)
metadata_filename = f'메타데이터_{str(start_date)[:10]}_{str(end_date)[:10]}.csv'
if df1.empty == 0:
    df1.to_csv(f'./{metadata_filename}', index=False, encoding='utf-8-sig')
print(f"메타데이터 파일을 저장했습니다: {metadata_filename}")

In [10]:
# 기사 본문 (전문), 기자 정보 추출하기

article_data_list = []

def fetch_article_data(article_url):
    resp = get(article_url)
    article_dom = BeautifulSoup(resp.text, 'html.parser')
    
    # 기사 제목 추출
    # title_tag = article_dom.select_one('h2#title_area')
    # title = title_tag.get_text(strip=True) if title_tag else ''

    # 기사 본문 추출
    content_tag = article_dom.select_one('article')
    content = preprocessing(content_tag.get_text(strip=True)) if content_tag else ''

    # 언론사 정보 추출
    # source_tag = article_dom.select_one('meta[property="og:article:author"]')
    # source = source_tag['content'] if source_tag else ''
    
    # 기자 정보 추출
    reporter_tag = article_dom.select_one('div.byline span')
    reporter = reporter_tag.get_text(strip=True) if reporter_tag else ''

    article_data = {
        # 'title': title,
        'link': article_url,
        'content': content,
        # 'source': source,
        'reporter': reporter
    }

    return article_data

In [11]:
for url in tqdm(list(df1['link'])):
    article_data = fetch_article_data(url)
    article_data_list.append(article_data)

100%|██████████| 193/193 [02:30<00:00,  1.29it/s]


In [12]:
df2 = pd.DataFrame(article_data_list)

## 데이터프레임 관리

In [13]:
# 데이터프레임 합치기
df = pd.merge(df1, df2)

# 데이터프레임 열 순서 바꾸기
df = df[['source', 'reporter', 'title', 'summary', 'content', 'link']]

# 중복 제거
df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)

# 열 이름 변경
df.columns = ['언론사', '기자', '제목', '기사요약', '기사전문', '기사링크']

# 데이터프레임 형태 확인
#df.head()

In [67]:
# CSV 파일로 저장
csv_filename = f'뉴스기사_{str(start_date)[:10]}_{str(end_date)[:10]}.csv'
df.to_csv(f'./{csv_filename}', index=False, encoding='utf-8-sig')

print(f"CSV 파일로 저장되었습니다: {csv_filename}")

CSV 파일로 저장되었습니다: 뉴스기사_2024-04-23_2024-04-24.csv


## 특정 언론사만 지정하여 라벨 남기기

In [21]:
df_select = df[(df['언론사']=='조선일보') | (df['언론사']=='중앙일보') | (df['언론사']=='동아일보') | (df['언론사']=='한겨레') | (df['언론사']=='경향신문') | (df['언론사']=='프레시안') | (df['언론사']=='오마이뉴스') | (df['언론사']=='문화일보')]

In [22]:
# 0 : 보수 , 1 : 진보
df_select['정치성향분류'] = [1 if x in ['한겨레', '경향신문', '프레시안', '오마이뉴스'] else 0 for x in df_select['언론사']]

C:\Users\infer\AppData\Local\Temp\ipykernel_13412\513145442.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_select['정치성향분류'] = [1 if x in ['한겨레', '경향신문', '프레시안', '오마이뉴스'] else 0 for x in df_select['언론사']]


In [23]:
df_select.head()

,언론사,기자,제목,기사요약,기사전문,기사링크,정치성향분류
3,중앙일보,정혜정 기자 jeong.hyejeong@joongang.co.kr,"조국 ""尹정권 심판하란 민심 확인…쎈 구호만 외치고 빠지지 않아""","조국 조국혁신당 대표는 23일 광주를 찾아 ""광주·전남 지역민이 보낸 지지는 우리 ...",조국 조국혁신당 대표가 23일 오후 광주 서구 김대중컨벤션센터에서 열린 '광주전남 ...,https://n.news.naver.com/mnews/article/025/000...,0
13,중앙일보,정혜정 기자 jeong.hyejeong@joongang.co.kr,"권영세, 尹 오찬 거절한 한동훈에 ""韓이 잘못…맞추는 게 예의""",권영세 국민의힘 의원은 23일 한동훈 전 국민의힘 비상대책위원장이 윤석열 대통령의 ...,윤석열 대통령이 지난 1월 29일 서울 용산 대통령실에서 한동훈 당시 국민의힘 비상...,https://n.news.naver.com/mnews/article/025/000...,0
21,조선일보,양지호 기자 yang.jiho@chosun.com,北 “핵 방아쇠”...계룡대 겨냥 ‘모의 핵탄두’ 장착해 시험 발사,북한은 22일 동해상으로 미사일 수 발을 발사한 것에 대해 “핵무기 종합 관리 체계...,북한이 지난 22일 김정은 국무위원장의 지도하에 핵 반격 가상 종합 전술 훈련을 실...,https://n.news.naver.com/mnews/article/023/000...,0
29,동아일보,김소영 동아닷컴 기자 sykim41@donga.com,"용산 떠나는 이관섭…尹대통령, 직접 차 문 여닫으며 배웅",윤석열 대통령이 23일 대통령실을 떠나는 이관섭 전 대통령비서실장을 끝까지 배웅했다...,윤석열 대통령이 23일 오후 용산 대통령실 청사에서 열린 이관섭 전 비서실장 퇴임 ...,https://n.news.naver.com/mnews/article/020/000...,0
41,프레시안,박세열 기자(ilys123@pressian.com),"윤상현 ""대참패 책임은 한동훈…내 제안 들었다면 132석 했을 것""",국민의힘의 수도권 참패 속 인천 동·미추홀을에서 당선된 윤상현 의원이 한동훈 전 비...,국민의힘의 수도권 참패 속 인천 동·미추홀을에서 당선된 윤상현 의원이 한동훈 전 비...,https://n.news.naver.com/mnews/article/002/000...,1
